In [2]:
import facebook
from datetime import datetime, timedelta
import pandas as pd

In [3]:
access_token = "EAACEdEose0cBAMOwkZBq2X9toGZCZCotBZBiaqdKqiAoMZCk0NbYZAwdoynb93bojM40qgeW5WBYSOINwiHegvmcb4JdXqoD4ElVFhx6ylIvawhJI2ut0HxYZBdYRZAOUiMRc8g1BncQjWzbVM83UuhxBvKyVhIXfPkOwx8j4rUqmXo46lGZBFwu1YMWhQH8CWQuTn1tzTmEHCAZDZD"
graph = facebook.GraphAPI(access_token, version="2.10")

In [4]:
"""Retrive post_id of a page which posted with chopped_days"""
def retrive_postsid(page_id, chopped_days):
    page_posts = graph.get_object('/'+str(page_id)+'/posts?')
    page_posts_used = []
    post_items = []
    while True:
        for post in page_posts['data']:
            created_time = datetime.strptime(post['created_time'][:19], '%Y-%m-%dT%H:%M:%S')
            chopped_time = datetime.now() - timedelta(days=chopped_days)
            if created_time>chopped_time:
                post_item = dict()
                post_item['post_id'] = post['id']
                post_item['page_id'] = page_id
                post_item['created_time'] = post['created_time'][:10]
                post_items.append(post_item)
                page_posts_used.append(post['id'])
            else:
                break
        if created_time<chopped_time:
            break
        if 'next' in page_posts['paging'].keys():
            after_key = page_posts['paging']['cursors']['after']
            page_posts = graph.get_object('/'+str(page_id)+'/posts?after='+after_key)
        else:
            break
    
    return page_posts_used, pd.DataFrame(post_items)

In [5]:
def retrive_reactions(posts_id, page_id):
    reaction_items = []
    i = 1
    for post_id in posts_id:
        print('retriving reactions')
        print(i, '/', len(posts_id) ,'At post_id: ', post_id, )
        i = i + 1
        post_reactions = graph.get_object('/'+str(post_id)+'/reactions')
        if bool(post_reactions['data']):    
            while True:
                for reaction in post_reactions['data']:
                    reaction_item = dict()
                    reaction_item['user_id'] = reaction['id']
                    reaction_item['post_id'] = post_id
                    reaction_item['page_id'] = page_id
                    reaction_item['reaction_type'] = reaction['type']
                    reaction_items.append(reaction_item)
                if 'next' in post_reactions['paging']:
                    after_key = post_reactions['paging']['cursors']['after']
                    post_reactions = graph.get_object('/'+str(post_id)+'/reactions?after='+str(after_key))
                else:
                    break
    print('finish retriving reactions!')
    return pd.DataFrame(reaction_items)  

In [6]:
def retrive_comments(posts_id, page_id):    
    comment_items = []
    i=1
    for post_id in posts_id:
        print('retriving comments')
        print(i, '/', len(posts_id) ,'At post_id: ', post_id, )
        i = i + 1
        post_comments = graph.get_object('/'+str(post_id)+'/comments')
        if bool(post_comments['data']):
            while True:
                for comment in post_comments['data']:
                    comment_item = dict()
                    comment_item['user_id'] = comment['from']['id']
                    comment_item['post_id'] = post_id
                    comment_item['page_id'] = '366743047016721'
                    comment_item['comment_body'] = comment['message']
                    comment_items.append(comment_item)
                if 'next' in post_comments['paging']:
                    after_key = post_comments['paging']['cursors']['after']
                    post_comments = graph.get_object('/'+str(post_id)+'/comments?after='+str(after_key))
                else:
                    break
    print('finish retriving comments!')
    return pd.DataFrame(comment_items) 

In [7]:
def retrive_pageid(page_name):
    return graph.get_object('/'+str(page_name))['id']

In [8]:
def retrive_data(page_name, chopped_days):
    from pathlib import Path
    comments_dir = Path('/data/comments/'+page_name+'.csv')
    page_id = retrive_pageid(page_name)
    posts_id, db_posts = retrive_postsid(page_id, chopped_days)
    db_posts.to_csv('./data/posts/'+page_name+'.csv')
    if comments_dir.exists():
        pass
    else:
        db_comments = retrive_comments(posts_id, page_id)
        db_comments.to_csv('./data/comments/'+page_name+'.csv')
    db_reactions = retrive_reactions(posts_id, page_id)
    db_reactions.to_csv('./data/reactions/'+page_name+'.csv') 
    return db_comments, db_reactions, db_posts

In [9]:
def retrive_performance(posts_id, page_id):
    rows = []
    for post_id in posts_id:
        if 'shares' in graph.get_object('/'+str(post_id)+'/?fields=shares'):
            num_reactions = graph.get_object('/'+str(post_id)+'/reactions?summary=1')['summary']['total_count'] 
            num_comments = graph.get_object('/'+str(post_id)+'/comments?summary=1')['summary']['total_count'] 
            num_shares = graph.get_object('/'+str(post_id)+'/?fields=shares')['shares']['count'] 
            row = dict()
            row['num_reactions'] = num_reactions
            row['num_comments'] = num_comments
            row['num_shares'] = num_shares
            row['page_id'] = page_id
            row['post_id'] = post_id
            rows.append(row)
    return pd.DataFrame(rows)

In [10]:
def retrive_overalldata(page_name, chopped_days):
    page_id = retrive_pageid(page_name)
    posts_id, _ = retrive_postsid(page_id, chopped_days)
    db_performance = retrive_performance(posts_id, page_id)
    db_performance.to_csv('./data/summary/'+page_name+'.csv')

In [11]:
def retrive_facebook_page(page_name):  
    _, _, _ = retrive_data(page_name, 180)
    retrive_overalldata(page_name, 90)
    db_summary_temp = pd.read_csv('./data/summary/'+page_name+'.csv')
    db_posts_temp = pd.read_csv('./data/posts/'+page_name+'.csv')
    db_posts_temp = db_posts_temp[['post_id', 'created_time']]
    temp = pd.merge(db_summary_temp, db_posts_temp, how='inner', on='post_id')
    temp.iloc[:,1:].to_csv('./data/summary2/'+page_name+'.csv')

In [26]:
retrive_facebook_page('daneating')

retriving comments
1 / 141 At post_id:  400267803487359_840971792750289
retriving comments
2 / 141 At post_id:  400267803487359_839759492871519
retriving comments
3 / 141 At post_id:  400267803487359_838142813033187
retriving comments
4 / 141 At post_id:  400267803487359_836844329829702
retriving comments
5 / 141 At post_id:  400267803487359_836775386503263
retriving comments
6 / 141 At post_id:  400267803487359_836499169864218
retriving comments
7 / 141 At post_id:  400267803487359_836327479881387
retriving comments
8 / 141 At post_id:  400267803487359_836300109884124
retriving comments
9 / 141 At post_id:  400267803487359_836082353239233
retriving comments
10 / 141 At post_id:  400267803487359_835641466616655
retriving comments
11 / 141 At post_id:  400267803487359_835160016664800
retriving comments
12 / 141 At post_id:  400267803487359_834625986718203
retriving comments
13 / 141 At post_id:  400267803487359_834252260088909
retriving comments
14 / 141 At post_id:  400267803487359_833

retriving comments
114 / 141 At post_id:  400267803487359_766285936885542
retriving comments
115 / 141 At post_id:  400267803487359_765284500319019
retriving comments
116 / 141 At post_id:  400267803487359_764803647033771
retriving comments
117 / 141 At post_id:  400267803487359_763706457143490
retriving comments
118 / 141 At post_id:  400267803487359_763120750535394
retriving comments
119 / 141 At post_id:  400267803487359_761640260683443
retriving comments
120 / 141 At post_id:  400267803487359_761091814071621
retriving comments
121 / 141 At post_id:  400267803487359_761004407413695
retriving comments
122 / 141 At post_id:  400267803487359_760811054099697
retriving comments
123 / 141 At post_id:  400267803487359_759627774218025
retriving comments
124 / 141 At post_id:  400267803487359_758601230987346
retriving comments
125 / 141 At post_id:  400267803487359_758480717666064
retriving comments
126 / 141 At post_id:  400267803487359_758011537712982
retriving comments
127 / 141 At post_i

retriving reactions
84 / 141 At post_id:  400267803487359_782381891942613
retriving reactions
85 / 141 At post_id:  400267803487359_782167865297349
retriving reactions
86 / 141 At post_id:  400267803487359_781847655329370
retriving reactions
87 / 141 At post_id:  400267803487359_780326488814820
retriving reactions
88 / 141 At post_id:  400267803487359_779965725517563
retriving reactions
89 / 141 At post_id:  400267803487359_778754238972045
retriving reactions
90 / 141 At post_id:  400267803487359_778557372325065
retriving reactions
91 / 141 At post_id:  400267803487359_778311765682959
retriving reactions
92 / 141 At post_id:  400267803487359_777830732397729
retriving reactions
93 / 141 At post_id:  400267803487359_776924559155013
retriving reactions
94 / 141 At post_id:  400267803487359_776381902542612
retriving reactions
95 / 141 At post_id:  400267803487359_775823079265161
retriving reactions
96 / 141 At post_id:  400267803487359_775750079272461
retriving reactions
97 / 141 At post_i

FileNotFoundError: File b'./data/summary/daneating.csv' does not exist

In [30]:
page_name = 'daneating'
retrive_overalldata(page_name, 90)
db_summary_temp = pd.read_csv('./data/summary/'+page_name+'.csv')
db_posts_temp = pd.read_csv('./data/posts/'+page_name+'.csv')
db_posts_temp = db_posts_temp[['post_id', 'created_time']]
temp = pd.merge(db_summary_temp, db_posts_temp, how='inner', on='post_id')
temp.iloc[:,1:].to_csv('./data/summary2/'+page_name+'.csv')

In [33]:
retrive_facebook_page('onnbaby')

retriving comments
1 / 472 At post_id:  161818843843444_1904900722868572
retriving comments
2 / 472 At post_id:  161818843843444_1902069813151663
retriving comments
3 / 472 At post_id:  161818843843444_1898956773462967
retriving comments
4 / 472 At post_id:  161818843843444_1898938276798150
retriving comments
5 / 472 At post_id:  161818843843444_1895627283795916
retriving comments
6 / 472 At post_id:  161818843843444_1895413310483980
retriving comments
7 / 472 At post_id:  161818843843444_1895344457157532
retriving comments
8 / 472 At post_id:  161818843843444_1895044887187489
retriving comments
9 / 472 At post_id:  161818843843444_1894199197272058
retriving comments
10 / 472 At post_id:  161818843843444_1894166040608707
retriving comments
11 / 472 At post_id:  161818843843444_1893961017295876
retriving comments
12 / 472 At post_id:  161818843843444_1892837367408241
retriving comments
13 / 472 At post_id:  161818843843444_1892473867444591
retriving comments
14 / 472 At post_id:  161818

retriving comments
112 / 472 At post_id:  161818843843444_1843094552382523
retriving comments
113 / 472 At post_id:  161818843843444_1843949065630405
retriving comments
114 / 472 At post_id:  161818843843444_1843842372307741
retriving comments
115 / 472 At post_id:  161818843843444_1843678332324145
retriving comments
116 / 472 At post_id:  161818843843444_1843078472384131
retriving comments
117 / 472 At post_id:  161818843843444_1842856299073015
retriving comments
118 / 472 At post_id:  161818843843444_1842841155741196
retriving comments
119 / 472 At post_id:  161818843843444_1841849429173702
retriving comments
120 / 472 At post_id:  161818843843444_1839461076079204
retriving comments
121 / 472 At post_id:  161818843843444_1841389025886409
retriving comments
122 / 472 At post_id:  161818843843444_1841276502564328
retriving comments
123 / 472 At post_id:  161818843843444_1840426849315960
retriving comments
124 / 472 At post_id:  161818843843444_1840405762651402
retriving comments
125 / 

retriving comments
222 / 472 At post_id:  161818843843444_1801583436533635
retriving comments
223 / 472 At post_id:  161818843843444_1801395943219051
retriving comments
224 / 472 At post_id:  161818843843444_1801165653242080
retriving comments
225 / 472 At post_id:  161818843843444_1800292926662686
retriving comments
226 / 472 At post_id:  161818843843444_1800214173337228
retriving comments
227 / 472 At post_id:  161818843843444_1800174380007874
retriving comments
228 / 472 At post_id:  161818843843444_1800052760020036
retriving comments
229 / 472 At post_id:  161818843843444_1799951530030159
retriving comments
230 / 472 At post_id:  161818843843444_1799874703371175
retriving comments
231 / 472 At post_id:  161818843843444_1799818300043482
retriving comments
232 / 472 At post_id:  161818843843444_1798999180125394
retriving comments
233 / 472 At post_id:  161818843843444_1798698343488811
retriving comments
234 / 472 At post_id:  161818843843444_1796741210351191
retriving comments
235 / 

retriving comments
332 / 472 At post_id:  161818843843444_1748536435171669
retriving comments
333 / 472 At post_id:  161818843843444_1748342628524383
retriving comments
334 / 472 At post_id:  161818843843444_1746028178755828
retriving comments
335 / 472 At post_id:  161818843843444_1745506625474650
retriving comments
336 / 472 At post_id:  161818843843444_1745249788833667
retriving comments
337 / 472 At post_id:  161818843843444_1744394888919157
retriving comments
338 / 472 At post_id:  161818843843444_1744359202256059
retriving comments
339 / 472 At post_id:  161818843843444_1742577662434213
retriving comments
340 / 472 At post_id:  161818843843444_1743027905722522
retriving comments
341 / 472 At post_id:  161818843843444_1742904502401529
retriving comments
342 / 472 At post_id:  161818843843444_1742787919079854
retriving comments
343 / 472 At post_id:  161818843843444_1741653835859929
retriving comments
344 / 472 At post_id:  161818843843444_1741319479226698
retriving comments
345 / 

retriving comments
442 / 472 At post_id:  161818843843444_1695044470520866
retriving comments
443 / 472 At post_id:  161818843843444_1695005010524812
retriving comments
444 / 472 At post_id:  161818843843444_1694638003894846
retriving comments
445 / 472 At post_id:  161818843843444_1694062540619059
retriving comments
446 / 472 At post_id:  161818843843444_1693697713988875
retriving comments
447 / 472 At post_id:  161818843843444_1693639807327999
retriving comments
448 / 472 At post_id:  161818843843444_1692843047407675
retriving comments
449 / 472 At post_id:  161818843843444_1692824120742901
retriving comments
450 / 472 At post_id:  161818843843444_1692230304135616
retriving comments
451 / 472 At post_id:  161818843843444_1690975964261050
retriving comments
452 / 472 At post_id:  161818843843444_1691178460907467
retriving comments
453 / 472 At post_id:  161818843843444_1690013151023998
retriving comments
454 / 472 At post_id:  161818843843444_1688522187839761
retriving comments
455 / 

retriving reactions
79 / 472 At post_id:  161818843843444_1856094521082526
retriving reactions
80 / 472 At post_id:  161818843843444_1856315167727128
retriving reactions
81 / 472 At post_id:  161818843843444_1856286124396699
retriving reactions
82 / 472 At post_id:  161818843843444_1856121317746513
retriving reactions
83 / 472 At post_id:  161818843843444_1856037891088189
retriving reactions
84 / 472 At post_id:  161818843843444_1855057547852890
retriving reactions
85 / 472 At post_id:  161818843843444_1855082227850422
retriving reactions
86 / 472 At post_id:  161818843843444_1854861201205858
retriving reactions
87 / 472 At post_id:  161818843843444_1854018137956831
retriving reactions
88 / 472 At post_id:  161818843843444_1853771577981487
retriving reactions
89 / 472 At post_id:  161818843843444_1853691307989514
retriving reactions
90 / 472 At post_id:  161818843843444_1853121808046464
retriving reactions
91 / 472 At post_id:  161818843843444_1851513238207321
retriving reactions
92 / 

retriving reactions
188 / 472 At post_id:  161818843843444_1817905658234746
retriving reactions
189 / 472 At post_id:  161818843843444_1817832704908708
retriving reactions
190 / 472 At post_id:  161818843843444_1817716951586950
retriving reactions
191 / 472 At post_id:  161818843843444_1817534001605245
retriving reactions
192 / 472 At post_id:  161818843843444_1816866115005367
retriving reactions
193 / 472 At post_id:  161818843843444_1816783751680270
retriving reactions
194 / 472 At post_id:  161818843843444_1816527728372539
retriving reactions
195 / 472 At post_id:  161818843843444_1816483938376918
retriving reactions
196 / 472 At post_id:  161818843843444_1816479145044064
retriving reactions
197 / 472 At post_id:  161818843843444_1815432688482043
retriving reactions
198 / 472 At post_id:  161818843843444_1815179705174008
retriving reactions
199 / 472 At post_id:  161818843843444_1815100515181927
retriving reactions
200 / 472 At post_id:  161818843843444_1814272628598049
retriving re

retriving reactions
296 / 472 At post_id:  161818843843444_1765831886775457
retriving reactions
297 / 472 At post_id:  161818843843444_1764778143547498
retriving reactions
298 / 472 At post_id:  161818843843444_1764514033573909
retriving reactions
299 / 472 At post_id:  161818843843444_1763448677013778
retriving reactions
300 / 472 At post_id:  161818843843444_1762902497068396
retriving reactions
301 / 472 At post_id:  161818843843444_1762082690483710
retriving reactions
302 / 472 At post_id:  161818843843444_1761835710508408
retriving reactions
303 / 472 At post_id:  161818843843444_1761624667196179
retriving reactions
304 / 472 At post_id:  161818843843444_1760985743926738
retriving reactions
305 / 472 At post_id:  161818843843444_1760709693954343
retriving reactions
306 / 472 At post_id:  161818843843444_1760350727323573
retriving reactions
307 / 472 At post_id:  161818843843444_1759584580733521
retriving reactions
308 / 472 At post_id:  161818843843444_1759425210749458
retriving re

retriving reactions
404 / 472 At post_id:  161818843843444_1708959849129328
retriving reactions
405 / 472 At post_id:  161818843843444_1708362325855747
retriving reactions
406 / 472 At post_id:  161818843843444_1707311505960829
retriving reactions
407 / 472 At post_id:  161818843843444_1706869489338364
retriving reactions
408 / 472 At post_id:  161818843843444_1707095599315753
retriving reactions
409 / 472 At post_id:  161818843843444_1707025895989390
retriving reactions
410 / 472 At post_id:  161818843843444_1706695056022474
retriving reactions
411 / 472 At post_id:  161818843843444_1706044582754188
retriving reactions
412 / 472 At post_id:  161818843843444_1705979432760703
retriving reactions
413 / 472 At post_id:  161818843843444_1705814706110509
retriving reactions
414 / 472 At post_id:  161818843843444_1705799816111998
retriving reactions
415 / 472 At post_id:  161818843843444_1705643799460933
retriving reactions
416 / 472 At post_id:  161818843843444_1704857876206192
retriving re

In [ ]:
retrive_facebook_page('ginmaiyood')

retriving comments
1 / 387 At post_id:  456836437854289_717442618460335
retriving comments
2 / 387 At post_id:  456836437854289_716830238521573
retriving comments
3 / 387 At post_id:  456836437854289_716647888539808
retriving comments
4 / 387 At post_id:  456836437854289_716598725211391
retriving comments
5 / 387 At post_id:  456836437854289_715939205277343
retriving comments
6 / 387 At post_id:  456836437854289_715919518612645
retriving comments
7 / 387 At post_id:  456836437854289_715776508626946
retriving comments
8 / 387 At post_id:  456836437854289_715558688648728
retriving comments
9 / 387 At post_id:  456836437854289_715411301996800
retriving comments
10 / 387 At post_id:  456836437854289_715157415355522
retriving comments
11 / 387 At post_id:  456836437854289_715012088703388
retriving comments
12 / 387 At post_id:  456836437854289_714788705392393
retriving comments
13 / 387 At post_id:  456836437854289_714636342074296
retriving comments
14 / 387 At post_id:  456836437854289_714

retriving comments
114 / 387 At post_id:  456836437854289_699131563624774
retriving comments
115 / 387 At post_id:  456836437854289_699146213623309
retriving comments
116 / 387 At post_id:  456836437854289_699085286962735
retriving comments
117 / 387 At post_id:  456836437854289_698740630330534
retriving comments
118 / 387 At post_id:  456836437854289_698962453641685
retriving comments
119 / 387 At post_id:  456836437854289_698693057001958
retriving comments
120 / 387 At post_id:  456836437854289_698652920339305
retriving comments
121 / 387 At post_id:  456836437854289_698540537017210
retriving comments
122 / 387 At post_id:  456836437854289_697823010422296
retriving comments
123 / 387 At post_id:  456836437854289_698183827052881
retriving comments
124 / 387 At post_id:  456836437854289_698032727067991
retriving comments
125 / 387 At post_id:  456836437854289_697794300425167
retriving comments
126 / 387 At post_id:  456836437854289_696757180528879
retriving comments
127 / 387 At post_i

retriving comments
225 / 387 At post_id:  456836437854289_686393858231878
retriving comments
226 / 387 At post_id:  456836437854289_686088248262439
retriving comments
227 / 387 At post_id:  456836437854289_686062658264998
retriving comments
228 / 387 At post_id:  456836437854289_685324141672183
retriving comments
229 / 387 At post_id:  456836437854289_685147205023210
retriving comments
230 / 387 At post_id:  456836437854289_685653118305952
retriving comments
231 / 387 At post_id:  456836437854289_685620171642580
retriving comments
232 / 387 At post_id:  456836437854289_685140658357198
retriving comments
233 / 387 At post_id:  456836437854289_684008135137117
retriving comments
234 / 387 At post_id:  456836437854289_685236601680937
retriving comments
235 / 387 At post_id:  456836437854289_684873005050630
retriving comments
236 / 387 At post_id:  456836437854289_685099395027991
retriving comments
237 / 387 At post_id:  456836437854289_683921868479077
retriving comments
238 / 387 At post_i

In [15]:
page_name = 'kintampro'
page_id = retrive_pageid(page_name)
posts_id, db_posts = retrive_postsid(page_id, 180)
db_reactions = retrive_reactions(posts_id, page_id)
db_reactions.to_csv('./data/reactions/'+page_name+'.csv') 
retrive_overalldata(page_name, 90)
db_summary_temp = pd.read_csv('./data/summary/'+page_name+'.csv')
db_posts_temp = pd.read_csv('./data/posts/'+page_name+'.csv')
db_posts_temp = db_posts_temp[['post_id', 'created_time']]
temp = pd.merge(db_summary_temp, db_posts_temp, how='inner', on='post_id')
temp.iloc[:,1:].to_csv('./data/summary2/'+page_name+'.csv')

retriving reactions
1 / 146 At post_id:  1663369887236712_1977895295784168
retriving reactions
2 / 146 At post_id:  1663369887236712_1977072192533145
retriving reactions
3 / 146 At post_id:  1663369887236712_1977047165868981
retriving reactions
4 / 146 At post_id:  1663369887236712_1977019412538423
retriving reactions
5 / 146 At post_id:  1663369887236712_1977000775873620
retriving reactions
6 / 146 At post_id:  1663369887236712_1976954039211627
retriving reactions
7 / 146 At post_id:  1663369887236712_1976600582580306
retriving reactions
8 / 146 At post_id:  1663369887236712_1976098652630499
retriving reactions
9 / 146 At post_id:  1663369887236712_1975650339341997
retriving reactions
10 / 146 At post_id:  1663369887236712_1975191272721237
retriving reactions
11 / 146 At post_id:  1663369887236712_1975133969393634
retriving reactions
12 / 146 At post_id:  1663369887236712_1974577479449283
retriving reactions
13 / 146 At post_id:  1663369887236712_1974028629504168
retriving reactions
1

retriving reactions
109 / 146 At post_id:  1663369887236712_1929905883916443
retriving reactions
110 / 146 At post_id:  1663369887236712_1927976020776096
retriving reactions
111 / 146 At post_id:  1663369887236712_1927880040785694
retriving reactions
112 / 146 At post_id:  1663369887236712_1926503294256702
retriving reactions
113 / 146 At post_id:  1663369887236712_1925467344360297
retriving reactions
114 / 146 At post_id:  1663369887236712_1924413954465636
retriving reactions
115 / 146 At post_id:  1663369887236712_1924316317808733
retriving reactions
116 / 146 At post_id:  1663369887236712_1923100504596981
retriving reactions
117 / 146 At post_id:  1663369887236712_1922500411323657
retriving reactions
118 / 146 At post_id:  1663369887236712_1922070361366662
retriving reactions
119 / 146 At post_id:  1663369887236712_1921570638083301
retriving reactions
120 / 146 At post_id:  1663369887236712_1920401574866874
retriving reactions
121 / 146 At post_id:  1663369887236712_1918036168436748

In [12]:
retrive_facebook_page('friendguha')

finish retriving comments!
finish retriving reactions!


KeyError: "['post_id' 'created_time'] not in index"

In [16]:
def retrive_facebook_page_1(page_name):  
    _, _, _ = retrive_data(page_name, 20000)
    retrive_overalldata(page_name, 20000)
    db_summary_temp = pd.read_csv('./data/summary/'+page_name+'.csv')
    db_posts_temp = pd.read_csv('./data/posts/'+page_name+'.csv')
    db_posts_temp = db_posts_temp[['post_id', 'created_time']]
    temp = pd.merge(db_summary_temp, db_posts_temp, how='inner', on='post_id')
    temp.iloc[:,1:].to_csv('./data/summary2/'+page_name+'.csv')

In [17]:
retrive_facebook_page_1('friendguha')

retriving comments
1 / 29 At post_id:  443407605824138_652537928244437
retriving comments
2 / 29 At post_id:  443407605824138_523942181104013
retriving comments
3 / 29 At post_id:  443407605824138_505001466331418
retriving comments
4 / 29 At post_id:  443407605824138_504507623047469
retriving comments
5 / 29 At post_id:  443407605824138_488420091322889
retriving comments
6 / 29 At post_id:  443407605824138_486782768153288
retriving comments
7 / 29 At post_id:  443407605824138_485163601648538
retriving comments
8 / 29 At post_id:  443407605824138_483343435163888
retriving comments
9 / 29 At post_id:  443407605824138_482381475260084
retriving comments
10 / 29 At post_id:  443407605824138_482228998608665
retriving comments
11 / 29 At post_id:  443407605824138_480733438758221
retriving comments
12 / 29 At post_id:  443407605824138_479960398835525
retriving comments
13 / 29 At post_id:  443407605824138_478985632266335
retriving comments
14 / 29 At post_id:  443407605824138_478418915656340
r